In [5]:
import numpy as np
import cv2
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

In [6]:
train_messy = "images/train/messy"
train_clean= "images/train/clean"
test_messy= "images/val/messy"
test_clean= "images/val/clean"
image_size = 128

In [28]:
def train_data():
    train_data_messy = [] 
    train_data_clean=[]
    for image1 in tqdm(os.listdir(train_messy)): 
        path = os.path.join(train_messy, image1)
        img1 = cv2.imread(path, cv2.IMREAD_GRAYSCALE) 
        img1 = cv2.resize(img1, (image_size, image_size))
        train_data_messy.append(img1) 
    for image2 in tqdm(os.listdir(train_clean)): 
        path = os.path.join(train_clean, image2)
        img2 = cv2.imread(path, cv2.IMREAD_GRAYSCALE) 
        img2 = cv2.resize(img2, (image_size, image_size))
        train_data_clean.append(img2) 
    
    train_data= np.concatenate((np.asarray(train_data_messy),np.asarray(train_data_clean)),axis=0)
    return train_data 

In [29]:
def test_data():
    test_data_messy = [] 
    test_data_clean=[]
    for image1 in tqdm(os.listdir(test_messy)): 
        path = os.path.join(test_messy, image1)
        img1 = cv2.imread(path, cv2.IMREAD_GRAYSCALE) 
        img1 = cv2.resize(img1, (image_size, image_size))
        test_data_messy.append(img1) 
    for image2 in tqdm(os.listdir(test_clean)): 
        if image2[0] == ".":
            continue
        path = os.path.join(test_clean, image2)
        img2 = cv2.imread(path, cv2.IMREAD_GRAYSCALE) 
        img2 = cv2.resize(img2, (image_size, image_size))
        test_data_clean.append(img2) 
    
    test_data= np.concatenate((np.asarray(test_data_messy),np.asarray(test_data_clean)),axis=0) 
    return test_data 

In [30]:
train_data = train_data() 
test_data = test_data()

100%|██████████| 11/11 [00:00<00:00, 192.00it/s]


In [31]:
x_data=np.concatenate((train_data,test_data),axis=0)
x_data = (x_data-np.min(x_data))/(np.max(x_data)-np.min(x_data))

In [32]:
z1 = np.zeros(96)
o1 = np.ones(96)
Y_train = np.concatenate((o1, z1), axis=0)
z = np.zeros(10)
o = np.ones(10)
Y_test = np.concatenate((o, z), axis=0)

In [33]:
y_data=np.concatenate((Y_train,Y_test),axis=0).reshape(x_data.shape[0],1)

In [34]:
x_data.shape,y_data.shape

((212, 128, 128), (212, 1))

In [37]:
X = x_data.reshape(x_data.shape[0],x_data.shape[1]*x_data.shape[2])
y = y_data
X.shape,y.shape

((212, 16384), (212, 1))

In [43]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [44]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.33, random_state=42)

In [45]:
param_grid = {
    'C': [0.1, 1, 10],  # Regularization parameter
    'kernel': ['linear', 'rbf'],  # Kernel type: linear or radial basis function
    'gamma': [0.1, 1, 'scale'],  # Kernel coefficient (scale for 1 / (n_features * X.var()))
}
svc = SVC()


In [47]:
grid_search = GridSearchCV(svc, param_grid, cv=5)  
grid_search.fit(X_train, y_train.ravel())  


GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10], 'gamma': [0.1, 1, 'scale'],
                         'kernel': ['linear', 'rbf']})

In [49]:
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_
best_params,best_model

({'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}, SVC(C=1))

In [50]:
best_model.score(X_test,y_test)

0.7285714285714285

In [51]:
class Dense_Layer:
    def __init__(self,ninputs,nneurons):
        # Initialising weights and biases
        self.weights = 0.01 * np.random.randn(ninputs,nneurons)
        self.biases = np.zeros((1,nneurons))
        
    # Forward Propagation    
    def forward(self,inputs):
        self.output = np.dot(inputs,self.weights) + self.biases
        self.inputs = inputs
    
    # Backpropagation
    def backward(self,dvalues):
        self.dinputs = np.dot(dvalues,self.weights.T)
        self.dweights = np.dot(self.inputs.T,dvalues)
        self.dbiases = np.sum(dvalues,axis=0,keepdims=True)

In [52]:
class Activation_Relu:
    def forward(self,inputs):
        self.output = np.maximum(0,inputs)
        self.inputs = inputs
    
    def backward(self,dvalues):
        self.dinputs = dvalues.copy()
        self.dinputs[self.inputs <= 0] = 0
        

In [53]:
class Activation_Softmax:
    def forward(self,inputs,y_true):
        expvals = np.exp(inputs - np.max(inputs, axis=1,
        keepdims=True) )
        self.output = expvals/np.sum(expvals,axis=1,keepdims=True)

In [54]:
class CrossEntropyLoss:
    def calculate(self,y_pred,y_true):
        # Turning one hot encoded values into sparse vectors
        if len(y_true.shape) == 2:
            y_true = np.argmax(y_true,axis=1)
        
        # Clipping to prevent log with 0
        ypred = np.clip(y_pred,1e-7,1-1e-7)
        
        return np.mean(-np.log(y_pred[range(len(y_pred)),y_true]))
    


In [55]:
class Activation_softmax_cross_entropy:
    def __init__(self):
        self.activation = Activation_Softmax()
        self.lossfunc = CrossEntropyLoss()
    
    def forward(self,inputs,y_true):
        self.activation.forward(inputs,y_true)
        self.output = self.activation.output
        return self.lossfunc.calculate(self.output,y_true)
    
    def backward(self,dvalues,y_true):
        samples = len(y_true)
        
        # Turning one hot encoded arrays to sparse vectors
        if len(y_true.shape) == 2:
            y_true = np.argmax(y_true,axis=1)
        
        self.dinputs = dvalues.copy()
        self.dinputs[range(samples),y_true] -= 1
        
        self.dinputs/=samples

In [56]:
class SGD_Optimizer:
    def __init__(self,lr=1,decay_rate=0.1,momentum=0.):
        self.initiallr = lr
        self.currentlr = lr
        self.decay_rate = decay_rate
        self.iterations = 0
        self.momentum = momentum
    
    def pre_update(self):
        self.currentlr = self.initiallr * (1/(1+(self.decay_rate * self.iterations)))

    def update_params(self,layer):
        
        if self.momentum:
            # if we don't have momentum arrays create them
            if not hasattr(layer, 'weight_momentums'):
                layer.weight_momentums = np.zeros_like(layer.weights)
                
                layer.bias_momentums = np.zeros_like(layer.biases)
            
            weight_updates = \
                self.momentum * layer.weight_momentums - self.currentlr * layer.dweights
            layer.weight_momentums = weight_updates
            
            bias_updates = \
                self.momentum * layer.bias_momentums - self.currentlr * layer.dbiases
            layer.bias_momentums = bias_updates
            
        else:
            weight_updates = -self.currentlr * layer.dweights
            bias_updates = -self.currentlr * layer.dbiases
                
        
        layer.weights += weight_updates
        layer.biases += bias_updates
        
    def post_update(self):
        self.iterations += 1

In [64]:
layer1 = Dense_Layer(16384,16)
act = Activation_Relu()
layer2 = Dense_Layer(16,2)
actloss = Activation_softmax_cross_entropy()
optimizer = SGD_Optimizer(1,decay_rate=1e-3,momentum=0.5)

for epoch in range(1001):
    layer1.forward(X_train)
    act.forward(layer1.output)
    layer2.forward(act.output)
    loss = actloss.forward(layer2.output,y_train)
    
    y_pred = np.argmax(actloss.output,axis=1)
    
    acc = np.mean(y_pred == y_train)
    if epoch % 100 == 0:
        print("Epoch : ",epoch, " Accuracy : ",acc, " Loss : ",loss)
    
    actloss.backward(actloss.output,y_train)
    layer2.backward(actloss.dinputs)
    act.backward(layer2.dinputs)
    layer1.backward(act.dinputs)

    optimizer.update_params(layer1)
    optimizer.update_params(layer2)

Epoch :  0  Accuracy :  0.4894862130529657  Loss :  0.6892998184760466
Epoch :  100  Accuracy :  0.4859154929577465  Loss :  9.647994453433246e-16
Epoch :  200  Accuracy :  0.4859154929577465  Loss :  9.647994453433246e-16
Epoch :  300  Accuracy :  0.4859154929577465  Loss :  9.647994453433246e-16
Epoch :  400  Accuracy :  0.4859154929577465  Loss :  9.647994453433246e-16
Epoch :  500  Accuracy :  0.4859154929577465  Loss :  9.647994453433246e-16
Epoch :  600  Accuracy :  0.4859154929577465  Loss :  9.647994453433246e-16
Epoch :  700  Accuracy :  0.4859154929577465  Loss :  9.647994453433246e-16
Epoch :  800  Accuracy :  0.4859154929577465  Loss :  9.647994453433246e-16
Epoch :  900  Accuracy :  0.4859154929577465  Loss :  9.647994453433246e-16
Epoch :  1000  Accuracy :  0.4859154929577465  Loss :  9.647994453433246e-16


In [63]:
y_train

array([[1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],